In [1]:
# Reimportar librerías después del reinicio del estado
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler,LabelEncoder
from sklearn.model_selection import train_test_split


Es menos exigente computacionalmente que Grid Search y Bayesian Optimization.
Encuentra buenas configuraciones sin necesidad de probar todas las combinaciones.
Es fácil de paralelizar (puedes ejecutar varias pruebas en diferentes threads de CPU).

In [2]:
df= pd.read_csv("/Users/isaromobru/Desktop/FakeNews 10.57.24/proyecto_machine_learning_Fake_News/data/processed/archivo.csv")

In [3]:
le = LabelEncoder()
df['subject_encoded'] = le.fit_transform(df['subject_grouped'])


In [4]:
from scipy.sparse import csr_matrix, hstack
# Eliminar Na
df['text'] = df['text'].fillna("")

# Vectoriza la columna 'text'
vectorizer_text = TfidfVectorizer(max_features=5000)
X_text = vectorizer_text.fit_transform(df['text'])

# Vectoriza la columna 'title'
vectorizer_title = TfidfVectorizer(max_features=1000)
X_title = vectorizer_title.fit_transform(df['title'])

# Convierte la columna 'subject_encoded' a una matriz dispersa
subject_feature = csr_matrix(df['subject_encoded'].values.reshape(-1, 1))

# Combina las tres fuentes de información en una única matriz de características
X = hstack([subject_feature, X_title, X_text])

# Define la variable objetivo; en este ejemplo, se asume que la columna "label" es la etiqueta a predecir
y = df['label']

# Divide los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
import xgboost as xgb

# Definir los modelos con parámetros iniciales
rf = RandomForestClassifier(random_state=42, n_jobs=-1)
xgb_model = xgb.XGBClassifier(random_state=42, tree_method='hist', verbosity=1)  # 'hist' optimiza CPU
gb = GradientBoostingClassifier(random_state=42)

# Definir el grid de hiperparámetros
param_grid_rf = {
    'n_estimators': [100, 200],
    'max_depth': [10, 20],
    'min_samples_split': [2, 5]
}

param_grid_xgb = {
    'n_estimators': [100, 200],
    'max_depth': [6, 10],
    'learning_rate': [0.01, 0.1],
    'subsample': [0.8, 1.0],
    'reg_lambda': [1, 10]
}

param_grid_gb = {
    'n_estimators': [100, 200],
    'max_depth': [3, 5],
    'learning_rate': [0.01, 0.1]
}

# Aplicar GridSearchCV a cada modelo
grid_rf = GridSearchCV(rf, param_grid_rf, cv=3, scoring='accuracy', verbose=1, n_jobs=-1)
grid_xgb = GridSearchCV(xgb_model, param_grid_xgb, cv=3, scoring='accuracy', verbose=1, n_jobs=-1)
grid_gb = GridSearchCV(gb, param_grid_gb, cv=3, scoring='accuracy', verbose=1, n_jobs=-1)

# Entrenar cada modelo optimizado
print("Optimizing Random Forest...")
grid_rf.fit(X_train, y_train)

print("Optimizing XGBoost...")
grid_xgb.fit(X_train, y_train)

print("Optimizing Gradient Boosting...")
grid_gb.fit(X_train, y_train)

# Mostrar los mejores hiperparámetros
print("Best Random Forest:", grid_rf.best_params_)
print("Best XGBoost:", grid_xgb.best_params_)
print("Best Gradient Boosting:", grid_gb.best_params_)

# Evaluar los mejores modelos en el conjunto de prueba
from sklearn.metrics import accuracy_score

models = {
    "Random Forest": grid_rf.best_estimator_,
    "XGBoost": grid_xgb.best_estimator_,
    "Gradient Boosting": grid_gb.best_estimator_
}

for name, model in models.items():
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    print(f"{name} Accuracy: {acc:.4f}")


Optimizing Random Forest...
Fitting 3 folds for each of 8 candidates, totalling 24 fits


Python(54182) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(54183) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(54184) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(54185) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(54186) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(54187) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(54188) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(54189) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


Optimizing XGBoost...
Fitting 3 folds for each of 32 candidates, totalling 96 fits


KeyboardInterrupt: 

In [16]:
import numpy as np
import pandas as pd
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from scipy.stats import randint, uniform
from sklearn.metrics import accuracy_score


# -------------------------------------
# 1. Definir modelos y espacios de búsqueda
# -------------------------------------
models_params = {
    "RandomForest": {
        "model": RandomForestClassifier(random_state=42),
        "params": {
            "n_estimators": randint(50, 200),
            "max_depth": randint(5, 50),
            "min_samples_split": randint(2, 10),
            "min_samples_leaf": randint(1, 5)
        }
    },
    "GradientBoosting": {
        "model": GradientBoostingClassifier(random_state=42),
        "params": {
            "n_estimators": randint(50, 200),
            "learning_rate": uniform(0.01, 0.3),
            "max_depth": randint(3, 10),
            "min_samples_split": randint(2, 10),
            "min_samples_leaf": randint(1, 5)
        }
    },
    "XGBoost": {
        "model": XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42),
        "params": {
            "n_estimators": randint(50, 200),
            "learning_rate": uniform(0.01, 0.3),
            "max_depth": randint(3, 10),
            "min_child_weight": randint(1, 6),
            "gamma": uniform(0, 0.3),
            "subsample": uniform(0.5, 0.5),
            "colsample_bytree": uniform(0.5, 0.5)
        }
    }
}

# -------------------------------------
# 2. Aplicar Random Search para cada modelo
# -------------------------------------
best_models = {}
results = []

for model_name, mp in models_params.items():
    print(f"\n🔍 Optimizando {model_name} con RandomizedSearchCV...\n")
    
    # Configurar RandomizedSearchCV
    search = RandomizedSearchCV(
        mp["model"], mp["params"], 
        n_iter=20, cv=3, scoring="accuracy", 
        n_jobs=-1, random_state=42, verbose=1
    )
    
    # Entrenar la búsqueda
    search.fit(X_train, y_train)
    
    # Guardar mejor modelo y resultado
    best_models[model_name] = search.best_estimator_
    
    # Evaluar en conjunto de prueba
    y_pred = search.best_estimator_.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    
    # Guardar resultados
    results.append({
        "Modelo": model_name,
        "Mejores Hiperparámetros": search.best_params_,
        "Precisión en Test": accuracy
    })

# -------------------------------------
# 3. Mostrar resultados finales
# -------------------------------------
df_results = pd.DataFrame(results)
import ace_tools as tools
tools.display_dataframe_to_user(name="Resultados de Randomized Search", dataframe=df_results)



🔍 Optimizando RandomForest con RandomizedSearchCV...

Fitting 3 folds for each of 20 candidates, totalling 60 fits


Aquí tienes un código en Python que utiliza RandomizedSearchCV para optimizar los hiperparámetros de tres modelos de clasificación:

Random Forest (RandomForestClassifier)
Gradient Boosting (GradientBoostingClassifier)
XGBoost (XGBClassifier)
El código incluye:
✅ Random Search para encontrar los mejores hiperparámetros
✅ Comparación del desempeño de los modelos
✅ Paralelización con n_jobs=-1 para acelerar la búsqueda

🔹 Explicación del Código
Generamos un dataset sintético con make_classification() (puedes cambiarlo por tu dataset real).
Definimos tres modelos (Random Forest, Gradient Boosting, XGBoost) y su respectivo espacio de búsqueda de hiperparámetros.
Usamos RandomizedSearchCV para encontrar la mejor configuración para cada modelo.
Guardamos los mejores modelos y evaluamos su precisión en el conjunto de prueba.
Mostramos los resultados en una tabla con:
Modelo
Mejores hiperparámetros encontrados
Precisión en el conjunto de prueba

📌 Consideraciones para MacBook Air
Usamos n_jobs=-1 para aprovechar todos los núcleos disponibles en la CPU.
La cantidad de iteraciones en Random Search (n_iter=20) es un valor equilibrado para obtener buenos resultados sin sobrecargar el sistema.
Si notas que el entrenamiento es lento, puedes:
Reducir n_iter=10
Usar un dataset más pequeño
Usar cv=2 en lugar de cv=3 en RandomizedSearchCV


📊 ¿Qué modelo será mejor?
RandomForest suele ser rápido y robusto.
GradientBoosting es eficiente en datos estructurados.
XGBoost suele obtener la mejor precisión, pero es más costoso computacionalmente.
